In [18]:
import requests
import pandas as pd
import datetime
import os

def download_dji_simple():
    """
    Simple DJI download method (requires only requests and pandas)
    """
    print("Downloading Dow Jones Index data...")
    
    # Parameter settings
    symbol = "^DJI"
    start_date = "1992-01-02"  # Start date
    
    # Create data directory
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # Convert to timestamp
    try:
        start_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        start_ts = int(start_dt.timestamp())
    except:
        start_ts = 694224000  # Default timestamp for 1992-01-02
    
    end_ts = int(datetime.datetime.now().timestamp())
    
    # Yahoo API
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}"
    
    params = {
        "period1": start_ts,
        "period2": end_ts,
        "interval": "1d",  # Daily data
        "events": "history"
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        print("Sending request to Yahoo Finance API...")
        response = requests.get(url, params=params, headers=headers, timeout=30)
        
        if response.status_code == 200:
            print("Parsing response data...")
            data = response.json()
            
            # Extract data
            result = data['chart']['result'][0]
            timestamps = result['timestamp']
            quotes = result['indicators']['quote'][0]
            
            # Create DataFrame
            df = pd.DataFrame({
                'Open': quotes['open'],
                'High': quotes['high'],
                'Low': quotes['low'],
                'Volume': quotes['volume'],
                'Close': quotes['close'],
            }, index=pd.to_datetime(timestamps, unit='s'))
            
            # Clean data
            df = df.dropna()
            df = df.sort_index()
            
            print(f"✓ Successfully downloaded {len(df)} records")
            
            # Save to CSV
            filename = "data/DJI.csv"
            df.to_csv(filename)
            
            # Display statistics
            print("\nStatistics:")
            print(f"File location: {filename}")
            print(f"Time range: {df.index[0].date()} to {df.index[-1].date()}")
            print(f"Latest closing price: ${df['Close'].iloc[-1]:,.2f}")
            
            # Display recent data
            print("\nLatest 5 trading days data:")
            print(df.tail())
            
            return df
        else:
            print(f"✗ Request failed, status code: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"✗ Download failed: {e}")
        return None

# Direct execution
download_dji_simple()

Sending request to Yahoo Finance API...
Parsing response data...
✓ Successfully downloaded 8544 records

Statistics:
File location: data/DJI.csv
Time range: 1992-01-02 to 2025-12-05
Latest closing price: $47,954.99

Latest 5 trading days data:
                             Open          High           Low     Volume  \
2025-12-01 14:30:00  47580.851562  47676.031250  47271.109375  482190000   
2025-12-02 14:30:00  47416.910156  47597.109375  47263.921875  510900000   
2025-12-03 14:30:00  47371.621094  47969.640625  47371.621094  490380000   
2025-12-04 14:30:00  47888.160156  48049.718750  47692.960938  498740000   
2025-12-05 14:30:00  47879.601562  48133.539062  47871.511719  456100000   

                            Close  
2025-12-01 14:30:00  47289.328125  
2025-12-02 14:30:00  47474.460938  
2025-12-03 14:30:00  47882.898438  
2025-12-04 14:30:00  47850.941406  
2025-12-05 14:30:00  47954.988281  


,Open,High,Low,Volume,Close
1992-01-02 14:30:00,3152.100098,3172.629883,3139.310059,23550000,3172.399902
1992-01-03 14:30:00,3172.399902,3210.639893,3165.919922,23620000,3201.500000
1992-01-06 14:30:00,3201.500000,3213.330078,3191.860107,27280000,3200.100098
1992-01-07 14:30:00,3200.100098,3210.199951,3184.479980,25510000,3204.800049
1992-01-08 14:30:00,3204.800049,3229.199951,3185.820068,29040000,3203.899902
...,...,...,...,...,...
2025-12-01 14:30:00,47580.851562,47676.031250,47271.109375,482190000,47289.328125
2025-12-02 14:30:00,47416.910156,47597.109375,47263.921875,510900000,47474.460938
2025-12-03 14:30:00,47371.621094,47969.640625,47371.621094,490380000,47882.898438
2025-12-04 14:30:00,47888.160156,48049.718750,47692.960938,498740000,47850.941406
